In [1]:
# импортирование основных библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# отключение назойливых варнингов
import warnings
warnings.filterwarnings('ignore')

In [2]:
# класс для отбора признаков  https://eli5.readthedocs.io/en/latest/blackbox/permutation_importance.html
from eli5.sklearn import PermutationImportance
import eli5

In [32]:
from sklearn_evaluation import plot
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

In [4]:
red = pd.read_csv('winequality-red.csv', sep=';')
white = pd.read_csv('winequality-white.csv', sep=';')

red['color'] = 'red'
white['color'] = 'white'

df = pd.concat([red, white])
df.index = pd.RangeIndex(len(df))

# выполним dummy кодирование цвета
map_ = {'red': 0, 'white': 1}
df['color'] = df['color'].map(map_)

In [5]:
# напишем функцию для юыстрой оценки классификатора
# На вход подается классивикатор и обучающая выборка. Функция выводит msa на кроссвалидации и 
# accuracy для отложеной выборки. Возращает предсказанное и тестовое значения на отложеной выборке.
def evaluate_model(model, X, y):
    X, y = shuffle(X, y)
    print(cross_val_score(model, X, y, scoring='neg_mean_absolute_error', n_jobs=-1, cv=5))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    predict = np.round(predict)
    print('accuracy = ', accuracy_score(y_test, predict))
    return y_test, predict

In [6]:
X = df.drop(columns='quality').values
y = df['quality'].values

In [7]:
# перемешиваем
X, y = shuffle(X, y)

Сделаем некоторый baseline на линейной регрессии, хотя уже упоминали, что линейные модели вряд ли сработают хорошо. Будет решать задачу регрессии, промежуточной метрикой качества пока пользоваться mse.

In [8]:
lin_reg = LinearRegression(n_jobs=-1)

In [9]:
# получаем некоторый baseline
test, pred = evaluate_model(lin_reg, X, y)

[-0.57001015 -0.54686461 -0.58476143 -0.56020748 -0.58751476]
accuracy =  0.5251282051282051


В среднем по метрике mse получаем чуть выше 0,5. Accuracy тоже порядка 52%. Всё же использовать accuracy для конечного случая будет неверно. Явно, необходимо учитывать на сколько сильно модель ошибается, 2 путает с 9, или 6 с 7, это совершенно разные вещи.

Попробуем более сильную модель.

In [10]:
gb = GradientBoostingRegressor()

In [12]:
test, pred = evaluate_model(gb, X, y)

[-0.58407027 -0.57587135 -0.56866666 -0.55699771 -0.56575836]
accuracy =  0.5538461538461539


Модель градиентного бустинга из sklearn со стандартными параметрами на "сырых" данных дала прирост в accuracy на 3%.

### Preprocessing and Feature Selection

In [13]:
logs = ['residual sugar', 'free sulfur dioxide', 'sulphates', 'chlorides', 'fixed acidity',
        'volatile acidity']
sqrts = ['citric acid']
df[logs] = df[logs].apply(np.log)
df[sqrts] = df[sqrts].apply(np.sqrt)

In [14]:
scaler = StandardScaler()

In [15]:
X = df.drop(columns='quality').values
y = df['quality'].values
X, y = shuffle(X, y)

X_scaled = scaler.fit_transform(X)

In [18]:
gb = GradientBoostingRegressor()
test, pred = evaluate_model(gb, X_scaled, y)

[-0.57124111 -0.56875718 -0.57197133 -0.55410706 -0.55320852]
accuracy =  0.5697435897435897


Теперь мы получаем качество порядка 57%, опять же без какой либо настройки параметров алгоритма.

Посмотрим на значимость признаков в данной модели.

In [19]:
perm = PermutationImportance(gb).fit(X_scaled, y)
eli5.show_weights(perm, feature_names = list(df.drop(columns='quality')))

Weight,Feature
0.3705 ± 0.0192,alcohol
0.1569 ± 0.0059,volatile acidity
0.0911 ± 0.0061,free sulfur dioxide
0.0656 ± 0.0074,sulphates
0.0446 ± 0.0026,residual sugar
0.0318 ± 0.0029,total sulfur dioxide
0.0225 ± 0.0034,chlorides
0.0177 ± 0.0027,pH
0.0166 ± 0.0012,citric acid
0.0138 ± 0.0026,density


Интересно, что самым важным признаком является алкоголь. А самым не значимым цвет, хотя из eda было видно его явное значение. Так же нет тех признаков, убрав которые мы увеличили бы качество работы алгоритма.

Попробуем расширить наше признаковое пространство нелинейными коэффициентами.

In [20]:
pf = PolynomialFeatures(2)
X_scaled_pol = pf.fit_transform(X_scaled)
print(X.shape)

(6497, 12)


In [21]:
gb = GradientBoostingRegressor()
test, pred = evaluate_model(gb, X_scaled_pol, y)

[-0.54928215 -0.54405582 -0.55420945 -0.53053751 -0.53327118]
accuracy =  0.5651282051282052


In [23]:
perm = PermutationImportance(gb).fit(X_scaled_pol, y)
eli5.show_weights(perm, top=20)

Weight,Feature
0.3255 ± 0.0133,x11
0.0731 ± 0.0066,x2
0.0176 ± 0.0037,x69
0.0138 ± 0.0012,x6
0.0129 ± 0.0014,x73
0.0102 ± 0.0013,x31
0.0100 ± 0.0022,x10
0.0096 ± 0.0012,x36
0.0087 ± 0.0018,x66
0.0081 ± 0.0007,x4


При увеличении признакого пространства качество алгоритма почти не изменилось, но возрасло время расчёта, поэтому обойдёмся без полиномиальных признаков. Предлагается на этом этапе прерваться с предобработкой и отбором признаков(которого пока по сути и не было) и заняться тренировкой моделей.

### Отбор моделей

Для отбора моделей будем пользоваться GridSearch

In [24]:
params = {'learning_rate': [0.05, 0.1, 0.5], 'n_estimators': [300, 350, 400, 450],'max_depth': [7, 8, 9, 10]}

#          'min_samples_split': [2, 3, 4], 

In [25]:
gb = GradientBoostingRegressor()

In [26]:
gs = GridSearchCV(gb, param_grid=params, scoring='neg_mean_absolute_error', n_jobs=-1, cv=5, verbose=10)
gs.fit(X_scaled, y)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   20.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   30.4s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   40.1s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   54.7s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'learning_rate': [0.05, 0.1, 0.5], 'n_estimators': [300, 350, 400, 450], 'max_depth': [7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=10)

In [27]:
gs.best_params_

{'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 450}

In [35]:
gb_best = GradientBoostingRegressor(**gs.best_params_)

In [36]:
evaluate_model(gb_best, X_scaled, y)

[-0.56569933 -0.55953443 -0.55772639 -0.58016261 -0.55679578]
accuracy =  0.6466666666666666


(array([6, 5, 5, ..., 6, 5, 6], dtype=int64),
 array([6., 5., 5., ..., 6., 6., 5.]))

Теперь качество модели колеблется от 65 до 69 %. А это уже серьёзный прирост порядка 10 процентов. Тут оговоримся, почему качество может колебаться из-за сильной небалансировки классов.

Посмотрим как проявит себя SVM на различных ядрах.

In [37]:
params = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C':[0.01, 0.1, 1, 10, 100]}

In [38]:
svr = SVR()

In [39]:
gs = GridSearchCV(svr, param_grid=params, scoring='neg_mean_absolute_error', n_jobs=-1, cv=5, verbose=10)
gs.fit(X_scaled, y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   49.3s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  96 out of 100 | elapsed:  6.0min remaining:   15.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  7.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=10)

In [40]:
gs.best_params_

{'C': 1, 'kernel': 'rbf'}

In [42]:
svr_best = SVR(**gs.best_params_)

In [46]:
evaluate_model(svr_best, X_scaled, y)

[-0.56074922 -0.57066973 -0.57228175 -0.56151963 -0.55524863]
accuracy =  0.5728205128205128


(array([5, 5, 6, ..., 6, 6, 5], dtype=int64),
 array([6., 5., 6., ..., 6., 6., 6.]))

Качество svm алгоритма соизмеримо с моделью градиеного бустинга со стандартными параметрами. То есть гораздо ниже достигнутого на данный момент качества. Если было очевидно, что линейное ядро не "спасёт" алгоритм, надежда было хотя бы на rbf, но нет.

Итого на данный момент попробовав ансамбли деревьев и несколько линейных моделей, так же несправедливо забыв
про метрические алгоритмы(сомневаюсь что они тут сделают что-то более адекватное чем сейчас), можно с увереностью
сказать, что стоит продолжать работать именно с ансамблями моделей. Для этого воспользуемся моделью из catboost, но 
сделаем это в другом notebook.